In [71]:
%%file vote.se

data player[3](address, commit, choice, has_revealed)
data num_voters
data player_map[]
data reward
data timer_start

event Notice(x:str)

def init():
    #If tied, refund to all players
    self.num_voters = 0
    
def add_voter(voter_commitment):
    #prevents a max callstack exception
    if self.test_callstack() != 1: return(-1)

    if self.num_voters < 3 and msg.value >= 1000:
        self.reward = self.reward + msg.value
        self.player[self.num_voters].address = msg.sender
        self.player[self.num_voters].commit = voter_commitment
        self.num_voters = self.num_voters + 1
        self.player_map[msg.sender] = self.num_voters
        if msg.value - 1000 > 0:
            send(0, msg.sender, msg.value-1000)
        return(0)
    else:    
        if msg.value > 0 :
            # prevent unnecessary leakage of money
            send(0, msg.sender, msg.value)
        return(-1)

def open(choice, nonce):
    pass

def count_vote():
    log(type=Notice, self.player[0].choice)
    log(type=Notice, self.player[1].choice)
    log(type=Notice, self.player[2].choice)

#returns the balance to ensure funds were lost and won properly
def balance_check():
    log(self.storage["player1"].balance)
    log(self.storage["player2"].balance)
    log(self.storage["player3"].balance)

def test_callstack():
    return(1)

Overwriting vote.se


In [72]:
import serpent
from ethereum import tester, utils, abi
from sha3 import sha3_256
import sys
import struct
import binascii
import random

s = tester.state()
c = s.abi_contract('vote.se')

# Use default addresses for Alice and Bob
alice = tester.a0
alice_key = tester.k0
bob = tester.a1
bob_key = tester.k1
cindy = tester.a2
cindy_key = tester.k2

tobytearr = lambda n, L: [] if L == 0 else tobytearr(n / 256, L - 1)+[n % 256]
zfill = lambda s: (32-len(s))*'\x00' + s

choice1 = random.randint(0,2)
nonce1 = random.randint(0,2**32-1)
ch1 = ''.join(map(chr, tobytearr(choice1, 32)))
no1 = ''.join(map(chr, tobytearr(nonce1, 32)))
print("Alice chooses {} which is: {}").format(choice1, nonce1)

## Use Alice's address for the commitment
s1 = ''.join([zfill(alice), ch1, no1])
comm1 = utils.sha3(s1)

choice2 = random.randint(0,2)
nonce2 = random.randint(0,2**32-1)
ch2 = ''.join(map(chr, tobytearr(choice2, 32)))
no2 = ''.join(map(chr, tobytearr(nonce2, 32)))
print("Bob chooses {} which is: {}\n").format(choice2, nonce2)

## Use Bob's address for the commitment
s2 = ''.join([zfill(bob), ch2, no2])
comm2 = utils.sha3(s2)

choice3 = random.randint(0,2)
nonce3 = random.randint(0,2**32-1)
ch3 = ''.join(map(chr, tobytearr(choice3, 32)))
no3 = ''.join(map(chr, tobytearr(nonce3, 32)))
print("Bob chooses {} which is: {}\n").format(choice3, nonce3)

## Use Bob's address for the commitment
s3 = ''.join([zfill(cindy), ch3, no3])
comm3 = utils.sha3(s3)

o = c.add_voter(comm1, value=1000, sender=alice_key)
print("Alice Added: {}").format(o)

o = c.add_voter(comm2, value=1000, sender=bob_key)
print("Bob Added: {}\n").format(o)

o = c.add_voter(comm3, value=1000, sender=cindy_key)
print("Cindy Added: {}\n").format(o)

o = c.open(choice1,nonce1, sender=alice_key)
print("Open for Alice: {}").format(o)

o = c.open(choice2,nonce2, sender=bob_key)
print("Open for Bob: {}\n").format(o)

o = c.open(choice3,nonce3, sender=cindy_key)
print("Open for Cindy: {}\n").format(o)

s.mine(3)

#o = c.balance_check()

Alice chooses 1 which is: 743751349
Bob chooses 0 which is: 4020772184

Bob chooses 0 which is: 3215296682

Alice Added: 0
Bob Added: 0

Cindy Added: 0

Open for Alice: None
Open for Bob: None

Open for Cindy: None



In [73]:
c.count_vote()

TransactionFailed: 

TypeError: trace() takes at least 4 arguments (1 given)

In [45]:
#s.blocks[0].__dict__
s.blocks

[<Block(#0 497865db)>,
 <Block(#1 4f6c8cf7)>,
 <Block(#2 81dd5bad)>,
 <Block(#3 d7ba6809)>]

In [48]:
s.blocks[0].__dict__.keys()

['ether_delta',
 'logs',
 'uncles',
 'transactions',
 'log_listeners',
 'journal',
 'db',
 'transaction_count',
 '_get_transactions_cache',
 'header',
 'state',
 'ancestor_hashes',
 'env',
 'suicides',
 'refunds',
 '_cached_rlp',
 'config',
 'receipts',
 'caches']

In [54]:
list(s.blocks[0].transactions)

['\x01', '\x80', '\x03', '\x02', '\x05', '\x04', '\x06']

In [57]:
list(s.blocks[3].transactions)

[]